# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [10]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [11]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [12]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010 driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [13]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [14]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [15]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton, driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [16]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [17]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [18]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [19]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [21]:
import panel as pn
from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import datetime

# -------------------------------
# Zero-shot, One-shot, Few-shot Prompts
# -------------------------------
prompts = {
    "Zero-shot": """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- Use the provided schema only
- Return 'I do not know' if not possible
- Remember revenue = price * quantity
- Remember cost = supply_price * quantity

### Database Schema
CREATE TABLE products (...);  -- same as exercise
...
### Answer
[SQL]""",

    "One-shot": """### Example
[QUESTION]List all product names[/QUESTION]
[SQL]SELECT name FROM products;[/SQL]

### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]
- Follow example format
- Return 'I do not know' if impossible
- Revenue = price * quantity, Cost = supply_price * quantity

### Database Schema
CREATE TABLE products (...);
...
### Answer
[SQL]""",

    "Few-shot": """### Examples
[QUESTION]Total revenue for all products[/QUESTION]
[SQL]SELECT SUM(price*quantity) FROM products;[/SQL]

[QUESTION]List all customers in California[/QUESTION]
[SQL]SELECT name FROM customers WHERE address LIKE '%California%';[/SQL]

### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]
- Follow examples
- Return 'I do not know' if impossible
- Revenue = price * quantity, Cost = supply_price * quantity

### Database Schema
CREATE TABLE products (...);
...
### Answer
[SQL]"""
}

chat_history = []

# -------------------------------
# Helper Functions
# -------------------------------
def format_report():
    """Format report content for PDF"""
    lines = ["# SQL Prompt Engineering Report", "## Prompts Used", ""]
    for key, value in prompts.items():
        lines.append(f"### {key} Prompt")
        lines.append(f"{value}\n")
    lines.append("## Findings & Analysis")
    lines.append(
        "- Zero-shot is fast but may hallucinate or misinterpret subtle instructions.\n"
        "- One-shot helps guide format and often reduces errors.\n"
        "- Few-shot is the most reliable, showing patterns and instructions clearly.\n"
        "- Hallucinations usually occur when schema details are missing or ambiguous.\n"
        "- Learned the importance of structured examples to guide AI behavior.\n"
    )
    return "\n".join(lines)

def generate_pdf(event=None):
    """Generate PDF report"""
    filename = f"sql_prompt_report_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    doc = SimpleDocTemplate(filename, pagesize=LETTER)
    styles = getSampleStyleSheet()
    story = [Paragraph("SQL Prompt Engineering Report", styles['Title']), Spacer(1,12)]

    report_text = format_report()
    for line in report_text.split("\n"):
        if line.startswith("# "):
            story.append(Paragraph(line[2:], styles['Title']))
        elif line.startswith("## "):
            story.append(Paragraph(line[3:], styles['Heading2']))
        elif line.startswith("### "):
            story.append(Paragraph(line[4:], styles['Heading3']))
        else:
            story.append(Paragraph(line, styles['Normal']))
    doc.build(story)
    print(f"✅ PDF generated: {filename}")

# -------------------------------
# Widgets and Layout
# -------------------------------
button_pdf = pn.widgets.Button(name="Generate PDF Report", button_type="success")
button_pdf.on_click(generate_pdf)

dashboard = pn.Column(
    "# SQL Prompt Engineering Experiment",
    "This dashboard demonstrates Zero-shot, One-shot, Few-shot prompts for SQL query generation.",
    pn.pane.Markdown("### Prompts Preview"),
    pn.pane.Markdown("\n---\n".join([f"**{k}**\n{v}" for k,v in prompts.items()])),
    button_pdf
)

pn.serve(dashboard, show=True)


Launching server at http://localhost:59191


✅ PDF generated: sql_prompt_report_20250820_175253.pdf
